# Class and Object

## 改变对象的字符串显示

你想改变对象实例的打印或者显示输出，要改变一个实例的字符串表示，可重新定义它的str repr方法

In [1]:
class Pair:
    def __init__(self, x, y):
        self.x = x 
        self.y = y
        
    def __repr__(self): # 返回一个实例的代码表示形式，通常用来重新构造这个实例
        return 'Pair ({0.x!r}, {0.y!r})'.format(self) # 0.x 0.y self.x self.y
    
    def __str__(self): # 将实例转换为一个字符串
        return '({0.x!s}, {0.y!s})'.format(self)
    
    '''
    def __repr__(self):
        return 'Pair(%s, %s') % (self.x, self.y) # instead
    '''
    
p = Pair(3, 4)
p

Pair (3, 4)

In [2]:
print(p)

(3, 4)


In [3]:
print('p is {!r}'.format(p))

p is Pair (3, 4)


In [9]:
print('p is {}'.format(p))

p is (3, 4)


自定义 __repr__() 和 __str__() 通常是很好的习惯，因为它能简化调试和实例输出。 例如，如果仅仅只是打印输出或日志输出某个实例，那么程序员会看到实例更加详细与有用的信息

In [10]:
print('debug : p is [{0!r}]')

debug : p is [{0!r}]


## 自定义字符串的格式化

In [29]:
_formats = {
    'ymd' : '{d.year}-{d.month}-{d.day}',
    'mdy' : '{d.month}/{d.day}/{d.year}',
    'dmy' : '{d.day}/{d.month}/{d.year}'
    }

_formats_1 = {
    'ymd' : '{0.year}-{0.month}-{0.day}',
    'mdy' : '{0.month}/{0.day}/{0.year}',
    'dmy' : '{0.day}/{0.month}/{0.year}'
    }

class Date:
    
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day
        
    def __format__(self, code):
        if code == '':
            code = 'ymd'
        fmt = _formats[code]
        return fmt.format(d=self) # d等于实例
    
d = Date(2012, 12, 21)

In [19]:
format(d)

'2012-12-21'

In [22]:
format(d, 'mdy')  # 2个参数 注意

('12/21/2012',)

In [23]:
'the date is {:ymd}'.format(d), 'the date is {:mdy}'.format(d)

('the date is 2012-12-21', 'the date is 12/21/2012')

__format__() 方法给Python的字符串格式化功能提供了一个钩子。 这里需要着重强调的是格式化代码的解析工作完全由类自己决定。因此，格式化代码可以是任何值。 例如，参考下面来自 datetime 模块中的代码

In [30]:
from datetime import date

d = date(2012, 12, 21)
format(d)

'2012-12-21'

In [31]:
format(d, '%A %B %d %Y')

'Friday December 21 2012'

In [32]:
'the end is {:%d %b %Y}. Goodbye'.format(d)

'the end is 21 Dec 2012. Goodbye'

## 让对象支持上下文管理协议

你想让你的对象支持上下文管理协议(with语句)

为了让一个对象兼容 with 语句，你需要实现 __enter__() 和 __exit__() 方法。 例如，考虑如下的一个类，它能为我们创建一个网络连接

In [34]:
# 这个类的关键特点在于它表示了一个网络连接，
# 但是初始化的时候并不会做任何事情(比如它并没有建立一个连接)。 连接的建立和关闭是使用with语句自动完成的
from socket import socket, AF_INET, SOCK_STREAM

class Lazyconnection:
    
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = family
        self.type = type
        self.sock = None
        
    def __enter__(self):
        if self.sock is not None:
            raise RuntimeError('Already connected')
        self.sock = socket(self.family, self.type)  # socket(AF_INET, SOCK_STREAM)
        self.sock.connect(self.address)  # socket(AF_INET, SOCK_STREAM).connect('127.0.0.1')
        return self.sock # 赋值给as声明的变量
    
    def __exit__(self, exc_ty, exc_val, tb):
        self.sock.close()
        self.sock = None

In [35]:
from functools import partial

conn = Lazyconnection(('www.python.org', 80)) # (())? address = ('www.python.org', 80) 
# Connection closed
with conn as s:
    # conn.__enter__() executes: connection open
    s.send(b'GET /index.html HTTP/1.0\r\n')
    s.send(b'Host: www.python.org\r\n')
    s.send(b'\r\n')
    resp = b''.join(iter(partial(s.recv, 8192), b''))
    # conn.__exit__() executes:connection closed

In [36]:
resp

b'HTTP/1.1 301 Moved Permanently\r\nServer: Varnish\r\nRetry-After: 0\r\nLocation: https://www.python.org/index.html\r\nContent-Length: 0\r\nAccept-Ranges: bytes\r\nDate: Sat, 03 Aug 2019 08:19:22 GMT\r\nVia: 1.1 varnish\r\nConnection: close\r\nX-Served-By: cache-hnd18746-HND\r\nX-Cache: HIT\r\nX-Cache-Hits: 0\r\nX-Timer: S1564820362.247628,VS0,VE0\r\nStrict-Transport-Security: max-age=63072000; includeSubDomains\r\n\r\n'

编写上下文管理器的主要原理是你的代码会放到 with 语句块中执行。 当出现 with 语句的时候，对象的 __enter__() 方法被触发， 它返回的值(如果有的话)会被赋值给 as 声明的变量。然后，with 语句块里面的代码开始执行。 最后，__exit__() 方法被触发进行清理工作

不管 with 代码块中发生什么，上面的控制流都会执行完，就算代码块中发生了异常也是一样的。 事实上，__exit__() 方法的第三个参数包含了异常类型、异常值和追溯信息(如果有的话)。 __exit__() 方法能自己决定怎样利用这个异常信息，或者忽略它并返回一个None值。 如果 __exit__() 返回 True ，那么异常会被清空，就好像什么都没发生一样， with 语句后面的程序继续在正常执行

还有一个细节问题就是 LazyConnection 类是否允许多个 with 语句来嵌套使用连
接。很显然，上面的定义中一次只能允许一个 socket 连接，如果正在使用一个 socket
的时候又重复使用 with 语句，就会产生一个异常了。不过你可以像下面这样修改下上
面的实现来解决这个问题：

In [38]:
from socket import socket, AF_INET, SOCK_STREAM

class Lazyconnection:
    
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = family
        self.type = type
        #self.sock = None
        self.connections = []
        
    def __enter__(self):
        #if self.sock is not None:
        #    raise RuntimeError('Already connected')
        sock = socket(self.family, self.type)
        #self.sock = socket(self.family, self.type)
        sock.connect(self.address)
        self.connection.append(sock)
        return sock
    
    def __exit__(self, exc_ty, exc_val, tb):
        #self.sock.close()
        #self.sock = None
        self.connections.pop().close()

In [39]:
# use case

from functools import partial

conn = Lazyconnection(('www.python.org', 8))
with conn as s1:
    pass
    with conn as s2:
        pass

TimeoutError: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。

在第二个版本中，LazyConnection 类可以被看做是某个连接工厂。在内部，一个列表被用来构造一个栈。 每次 __enter__() 方法执行的时候，它复制创建一个新的连接并将其加入到栈里面。 __exit__() 方法简单的从栈中弹出最后一个连接并关闭它。 这里稍微有点难理解，不过它能允许嵌套使用 with 语句创建多个连接，就如上面演示的那样。

在需要管理一些资源比如文件、网络连接和锁的编程环境中，使用上下文管理器是很普遍的。 这些资源的一个主要特征是它们必须被手动的关闭或释放来确保程序的正确运行。 例如，如果你请求了一个锁，那么你必须确保之后释放了它，否则就可能产生死锁。 通过实现 __enter__() 和 __exit__() 方法并使用 with 语句可以很容易的避免这些问题， 因为 __exit__() 方法可以让你无需担心这些了。

在 contextmanager 模块中有一个标准的上下文管理方案模板，可参考9.22小节。 同时在12.6小节中还有一个对本节示例程序的线程安全的修改版

## 创建大量对象时节省内存方法 - 低

你的程序要创建大量(可能上百万)的对象，导致占用很大的内存

对于主要是用来当成简单的数据结构的类而言，你可以通过给类添加 __slots__ 属性来极大的减少实例所占的内存

In [41]:
class Date:
    __slots__ = ['year', 'month', 'day'] # ? 
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

当你定义 __slots__ 后，Python就会为实例使用一种更加紧凑的内部表示。 实例通过一个很小的固定大小的数组来构建，而不是为每个实例定义一个字典，这跟元组或列表很类似。 在 __slots__ 中列出的属性名在内部被映射到这个数组的指定小标上。 使用slots一个不好的地方就是我们不能再给实例添加新的属性了，只能使用在 __slots__ 中定义的那些属性名

使用slots后节省的内存会跟存储属性的数量和类型有关。 不过，一般来讲，使用到的内存总量和将数据存储在一个元组中差不多。 为了给你一个直观认识，假设你不使用slots直接存储一个Date实例， 在64位的Python上面要占用428字节，而如果使用了slots，内存占用下降到156字节。 如果程序中需要同时创建大量的日期实例，那么这个就能极大的减小内存使用量了。

尽管slots看上去是一个很有用的特性，很多时候你还是得减少对它的使用冲动。 Python的很多特性都依赖于普通的基于字典的实现。 另外，定义了slots后的类不再支持一些普通类特性了，比如多继承。 大多数情况下，你应该只在那些经常被使用到的用作数据结构的类上定义slots (比如在程序中需要创建某个类的几百万个实例对象)。

关于 __slots__ 的一个常见误区是它可以作为一个封装工具来防止用户给实例增加新的属性。 尽管使用slots可以达到这样的目的，但是这个并不是它的初衷。 __slots__ 更多的是用来作为一个内存优化工具

## 在类中封装属性名

你想封装类的实例上面的“私有”数据，但是Python语言并没有访问控制

Python程序员不去依赖语言特性去封装数据，而是通过遵循一定的属性和方法命名规约来达到这个效果。 第一个约定是任何以单下划线_开头的名字都应该是内部实现

In [42]:
class A:
    
    def __init__(self):
        self._internal = 0 # an internal attribute
        self.public = 1 # a public attribute
        
    def pubilic_method(self):
        pass
    
    def _internal_method(self):
        pass

Python并不会真的阻止别人访问内部名称。但是如果你这么做肯定是不好的，可能会导致脆弱的代码。 同时还要注意到，使用下划线开头的约定同样适用于模块名和模块级别函数。 例如，如果你看到某个模块名以单下划线开头(比如_socket)，那它就是内部实现。 类似的，模块级别函数比如 sys._getframe() 在使用的时候就得加倍小心了。

你还可能会遇到在类定义中使用两个下划线(__)开头的命名

In [44]:
class B:
    
    def __init__(self):
        self.__private = 0
        
    def __private_method(self):
        pass
    
    def public_method(self):
        pass
        self.__private_method()

In [ ]:
#使用双下划线开始会导致访问名称变成其他形式。 
#比如，在前面的类B中，私有属性会被分别重命名为 _B__private 和 #_B__private_method
#这时候你可能会问这样重命名的目的是什么，答案就是继承——这种属性通过继承是无法被覆盖

class C(B):
    def __init__(self):
        super().__init__() # ?
        self.__private = 1 # Does not override B.__private
        
    # Dose  not override B.__private_method()
    def __private_method(self):
        pass
    
# 私有名称 __private 和 __private_method 被重命名为 _C__private 和 _C__private_method
#这个跟父类B中的名称是完全不同的

你应该让你的非公共名称以单下划线开头。但是，如果你清楚你的代码会涉及到子类， 并且有些内部属性应该在子类中隐藏起来，那么才考虑使用双下划线方案

## 创建可管理的属性

你想给某个实例attribute增加除访问与修改之外的其他处理逻辑，比如类型检查或合法性验证

自定义某个属性的一种简单方法是将它定义为一个property。 例如，下面的代码定义了一个property，增加对一个属性简单的类型检查

In [1]:
class Person:
    
    def __init__(self,first_name):
        self.first_name = first_name
        
    # Getter function
    @property
    def first_name(self): # 第一个方法是一个 getter 函数，它使得 first_name 成为一个属性。 
        return self._first_name # self._first_name?
    
    # Setter function
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value
        
    # Deleter function (optional)
    @first_name.deleter
    def first_name(self):
        raise AttributeError('Can"t delete attribute')
        
# 上述代码中有三个相关联的方法，这三个方法的名字都必须一样。 
# 其他两个方法给 first_name 属性添加了 setter 和 deleter 函数。 
# 需要强调的是只有在 first_name 属性被创建后， 后面的两个装饰器 @first_name.setter 和 @first_name.deleter 才能被定义

In [2]:
a = Person('Guido')
a.first_name

'Guido'

In [4]:
a.first_name='gang' # because attribute? not function

In [5]:
a.first_name

'gang'

In [6]:
del a.first_name

AttributeError: Can"t delete attribute

In [7]:
a.first_name

'gang'

In [32]:
class Person:
    
    def __init__(self,first_name):
        self.first_name = first_name
        
    def first_name(self): # 第一个方法是一个 getter 函数，它使得 first_name 成为一个属性。 
        return self._first_name # self._first_name?
    
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value
        
    def first_name(self):
        raise AttributeError('Can"t delete attribute')

In [33]:
a = Person('liugang')

In [34]:
a.first_name = 42

In [35]:
a.first_name = 'gang'

In [36]:
a.first_name

'gang'

In [42]:
#在实现一个property的时候，底层数据(如果有的话)仍然需要存储在某个地方。 
#因此，在get和set方法中，你会看到对 _first_name 属性的操作，这也是实际数据保存的地方。 
#另外，你可能还会问为什么 __init__() 方法中设置了 (self.first_name) 而不是 (self._first_name)  
#在这个例子中，我们创建一个property的目的就是在设置attribute的时候进行检查。 
#因此，你可能想在初始化的时候也进行这种类型检查。
#通过设置 self.first_name ，自动调用 setter 方法， 这个方法里面会进行参数的检查，
#否则.就是直接访问 self._first_name了

# 还能在已存在的get和set方法基础上定义property

class Person:
    def __init__(self, first_name):
        self.set_first_name(first_name)

    def get_first_name(self):
        return self._first_name
    
    def set_first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value
        
    def del_first_name(self):
        raise AttributeError('Can"t delete attribute')
        
    name = property(get_first_name, set_first_name, del_first_name)


In [43]:
Person.first_name.fget # ?

AttributeError: type object 'Person' has no attribute 'first_name'

In [20]:
p = Person()
p.get_first_name()

TypeError: __init__() missing 1 required positional argument: 'first_name'

In [ ]:
# 不要像下面这样写有大量重复代码的property定义
class Person:
    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name

    @property
    def first_name(self):
        return self._first_name

    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value

    # Repeated property code, but for a different name (bad!)
    @property
    def last_name(self):
        return self._last_name

    @last_name.setter
    def last_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._last_name = value

Properties 还是一种定义动态计算 attribute 的方法。这种类型的 attributes 并不会
被实际的存储，而是在需要的时候计算出来。

In [3]:
import math

class Circle:
    def __init__(self, radius):
        self.radius = radius
        
    @property
    def area(self):
        return math.pi * self.radius ** 2
    
    @property
    def diameter(self):
        return self.radius * 2
    
    @property
    def perimeter(self):
        return 2 * math.pi * self.radius

在这里，我们通过使用 properties，将所有的访问接口形式统一起来，对半径、直
径、周长和面积的访问都是通过属性访问，就跟访问简单的 attribute 是一样的。如果
不这样做的话，那么就要在代码中混合使用简单属性访问和方法调用。下面是使用的实
例：

In [4]:
c = Circle(4.0)
c.radius

4.0

In [5]:
c.area, c.perimeter

(50.26548245743669, 25.132741228718345)

尽管 properties 可以实现优雅的编程接口，但有些时候你还是会想直接使用 getter
和 setter 函数。例如：

In [6]:
p = Person('Guido')
p.get_first_name()

'Guido'

In [7]:
p.set_first_name('Laraau')

In [8]:
p.get_first_name()

'Laraau'

## 调用父类方法

你想在子类中调用父类的某个已经被覆盖的方法。

为了调用父类 (超类) 的一个方法，可以使用 super() 函数

In [9]:
class A:
    def spam(self):
        print('A.spam')
        
class B(A):
    def spam(self):
        print('B.spam')
        super().spam() # call parent spam()

In [10]:
a = A()
b = B()
b.spam()

B.spam
A.spam


super() 函数的一个常见用法是在 __init__() 方法中确保父类被正确的初始化
了：

In [12]:
class A:
    def __init__(self):
        self.x = 0
        
class B(A):
    def __init__(self):
        super().__init__()
        self.y = 1

In [13]:
a = A()
b = B()
a.x

0

In [14]:
b.x, b.y

(0, 1)

super() 的另外一个常见用法出现在覆盖 Python 特殊方法的代码中，比如：

In [15]:
class Proxy:
    def __init__(self, obj):
        self._obj = obj
        
    #Delegate attribute lookup to internal obj
    def __getattr__(self, name):
        return getattr(self._obj, name)
    
    #Delegate attribute assignment
    def __setattr__(self, name, value):
        if name.startswith('_'):
            super().__setattr__(name, value) # call original __setattr__
        else:
            setattr(self._obj, name, value)

在上面代码中，__setattr__() 的实现包含一个名字检查。如果某个属性名以下划
线 (_) 开头，就通过 super() 调用原始的 __setattr__() ，否则的话就委派给内部的
代理对象 self._obj 去处理。这看上去有点意思，因为就算没有显式的指明某个类的
父类，super() 仍然可以有效的工作

实际上，大家对于在 Python 中如何正确使用 super() 函数普遍知之甚少。你有时候会看到像下面这样直接调用父类的一个方法

In [44]:
class Base:
    def __init__(self):
        print('Base.__init__')
        
class A(Base):
    def __init__(self):
        Base.__init__(self) 
        print('A.__init__')

In [45]:
a = A()

Base.__init__
A.__init__


In [50]:
class Base:
    def __init__(self):
        print('Base.__init__')
        
class A(Base):
    def __init__(self):
        super().__init__()  # super.__init__(self)  error
        print('A.__init__')

In [51]:
a = A()

Base.__init__
A.__init__


在更复杂的涉及到多继承的代码中就有可能导致很奇怪的问题发生

In [18]:
class Base:
    def __init__(self):
        print("Base.__init__")
        
class A(Base):
    def __init__(self):
        Base.__init__(self)
        print('A.__init__')
        
class B(Base):
    def __init__(self):
        Base.__init__(self)
        print('B.__init__')
        
class C(A,B):
    def __init__(self):
        A.__init__(self)
        B.__init__(self)
        print('C.__init__')

In [19]:
c = C()

Base.__init__
A.__init__
Base.__init__
B.__init__
C.__init__


两次调用 Base.__init__() 没什么坏处，但有时候却不是。另一方面，假设你在代码中换成使用 super() ，结果就很完美了：

In [21]:
class Base:
    def __init__(self):
        print('Base.__init__')
        
class A(Base):
    def __init__(self):
        super().__init__() # Base.__init__(self)
        print('A.__init__')
        
class B(Base):
    def __init__(self):
        super().__init__()
        print('B.__init__')
        
class C(A, B):
    def __init__(self):
        super().__init__()
        print('C.__init__')

In [22]:
c = C()

Base.__init__
B.__init__
A.__init__
C.__init__


## review 2019.11.5 

为了弄清它的原理，我们需要花点时间解释下 Python 是如何实现继承的。对于你定义的每一个类，Python 会计算出一个所谓的方法解析顺序 (MRO) 列表。这个 MRO列表就是一个简单的所有基类的线性顺序表

In [24]:
C.__mro__ # 后续也太理解，可能实际并不常用

(__main__.C, __main__.A, __main__.B, __main__.Base, object)

为了实现继承，Python 会在 MRO 列表上从左到右开始查找基类，直到找到第一个匹配这个属性的类为止。

而这个 MRO 列表的构造是通过一个 C3 线性化算法来实现的。我们不去深究这个算法的数学原理，它实际上就是合并所有父类的 MRO 列表并遵循如下三条准则：

##### • 子类会先于父类被检查

##### • 多个父类会根据它们在列表中的顺序被检查

##### • 如果对下一个类存在两个合法的选择，选择第一个父类

老实说，你所要知道的就是 MRO 列表中的类顺序会让你定义的任意类层级关系变得有意义。

当你使用 super() 函数时，Python 会在 MRO 列表上继续搜索下一个类。只要每个重定义的方法统一使用 super() 并只调用它一次，那么控制流最终会遍历完整个MRO 列表，每个方法也只会被调用一次。这也是为什么在第二个例子中你不会调用两次 Base.__init__() 的原因。

super() 有个令人吃惊的地方是它并不一定去查找某个类在 MRO 中下一个直接父类，你甚至可以在一个没有直接父类的类中使用它。例如，考虑如下这个类：

In [25]:
class A:
    def spam(self):
        print('A.spam')
        super().spam()

In [26]:
a = A()
a.spam()

A.spam


AttributeError: 'super' object has no attribute 'spam'

但是，如果你使用多继承的话看看会发生什么：

In [27]:
class B:
    def spam(self):
        print('B.spam')
        
class C(A,B):
    pass

c = C()
c.spam()

A.spam
B.spam


你可以看到在类 A 中使用 super().spam() 实际上调用的是跟类 A 毫无关系的类B 中的 spam() 方法。这个用类 C 的 MRO 列表就可以完全解释清楚了：

In [28]:
C.__mro__

(__main__.C, __main__.A, __main__.B, object)

在定义混入类的时候这样使用 super() 是很普遍的

然而，由于 super() 可能会调用不是你想要的方法，你应该遵循一些通用原则。首先，确保在继承体系中所有相同名字的方法拥有可兼容的参数签名 (比如相同的参数个数和参数名称)。这样可以确保 super() 调用一个非直接父类方法时不会出错。其次，最好确保最顶层的类提供了这个方法的实现，这样的话在 MRO 上面的查找链肯定可以找到某个确定的方法

## 子类中扩展 property 没怎么懂

你想创建一个新的拥有一些额外功能的实例属性类型，比如类型检查。

如果你想创建一个全新的实例属性，可以通过一个描述器类的形式来定义它的功
能。下面是一个例子：

In [1]:
class Person:
    def __init__(self, name):
        self.name = name
        
    # Getter function
    @property
    def name(self):
        return self._name
    
    #Setter function
    @name.setter
    def name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._name = value
        
    #Deleter function
    @name.deleter  # deleter  ! r
    def name(self):
        raise AttributeError('Can"t delete attribute')

# 下面是一个实例类，它继承自Person并扩展了name属性的功能
class SubPerson(Person):
    @property
    def name(self):
        print('Getting name')
        return super().name
    
    @name.setter
    def name(self, value):
        print('Setting name to ', value)
        super(SubPerson, SubPerson).name.__set__(self, value)
        
    @name.deleter
    def name(self):
        print('Deleting name')
        super(SubPerson, SubPerson).name.__delete__(self)

In [2]:
s = SubPerson('Guido')

Setting name to  Guido


In [4]:
s.name 

Getting name


'Guido'

In [5]:
s.name = 'liugang'

Setting name to  liugang


In [6]:
s.name = 444

Setting name to  444


TypeError: Expected a string

如果你仅仅只想扩展 property 的某一个方法，那么可以像下面这样写

In [7]:
class SubPerson(Person):
    @Person.name.getter
    def name(self):
        print('Getting name')
        return super().name
    
# 你只想修改setter方法

class SubPerson(Person):
    @Person.name.setter
    def name(self, value):
        print('Setting name to', value)
        super(SubPerson, SubPerson).name.__set__(self, value)

.......

在子类中扩展一个 property 可能会引起很多不易察觉的问题，因为一个 property
其实是 getter、setter 和 deleter 方法的集合，而不是单个方法。因此，当你扩展一
个 property 的时候，你需要先确定你是否要重新定义所有的方法还是说只修改其中某
一个。

在第一个例子中，所有的 property 方法都被重新定义。在每一个方法中，使用了
super() 来调用父类的实现。在 setter 函数中使用 super(SubPerson, SubPerson).
name.__set__(self, value) 的语句是没有错的。为了委托给之前定义的 setter 方
法，需要将控制权传递给之前定义的 name 属性的 __set__() 方法。不过，获取这个
方法的唯一途径是使用类变量而不是实例变量来访问它。这也是为什么我们要使用
super(SubPerson, SubPerson) 的原因。

如果你只想重定义其中一个方法，那只使用 @property 本身是不够的。比如，下面
的代码就无法工作：

....................

## 创建新的类或实例属性

你想创建一个新的拥有一些额外功能的实例属性类型,比如类型检查

如果你想创建一个全新的实例属性，可以通过一个描述器类的形式来定义它的功
能。下面是一个例子

In [3]:
class Integer:
    def __init__(self, name):
        self.name = name
        
    def __get__(self, instance, cls): # name sui bian qi?
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]
        
    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise TypeError('Expected an int')
        instance.__dict__[self.name] = value # ?

    def __delete__(self, instance):
        del instance.__dict__[self.name]
        
'''
python 中__dict__存储了该对象的一些属性

类和实例分别拥有自己的__dict__

在__init__中声明的变量，会存到实例的__dict__中
'''

'\npython 中__dict__存储了该对象的一些属性\n\n类和实例分别拥有自己的__dict__\n\n在__init__中声明的变量，会存到实例的__dict__中\n'

In [16]:
class Point:
    x = Integer('x')
    y = Integer('y')
    
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [17]:
p = Point(2, 3)
p.x

2

In [18]:
p.y = 5
p.x = 2.3

TypeError: Expected an int

通过定义一个描述器，你可以在底层捕获核心的实例操作 (get, set, delete)，并且可完全自定义它们的行为。这是一个强大的工具，有了它你可以实现很多高级功能，并且它也是很多高级库和框架中的重要工具之一。

描述器的一个比较困惑的地方是它只能在类级别被定义，而不能为每个实例单独
定义。因此，下面的代码是无法工作的：

In [19]:
# Dose not work

class Point(object):
    def __init__(self, x, y):
        self.x = Integer('x')
        self.y = Integet('y')
        self.x = x
        self.y = y

In [22]:
class Integer:
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]

In [23]:
p = Point(2, 3)
p.x

TypeError: Integer() takes no arguments

In [24]:
Point.x

AttributeError: type object 'Point' has no attribute 'x'

描述器通常是那些使用到装饰器或元类的大型框架中的一个组件。同时它们的使
用也被隐藏在后面。举个例子，下面是一些更高级的基于描述器的代码，并涉及到一个
类装饰器：

In [25]:
# Descriptor for a type-checked attribute
class Typed:
    def __init__(self, name, expected_type):
        self.name = name
        self.expected_type = expected_type
        
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]
        
    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError('Expected ' + str(self.expected_type))
        instance.__dict__[self.name] = value
        
    def __delete__(self, instance):
        del instance.__dict__[self.name]
        
# Class decorator that applies it to selected attributes
def typeassert(**kwargs):
    def decorate(cls):
        for name, expected_type in kwargs.items():
            # Attach a Typed descriptor to the class
            setattr(cls, name, Typed(name, expected_type))
        return cls
    return decorate

# Example use
@typeassert(name=str, shares=int, price=float)
class Stock:
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

最后要指出的一点是，如果你只是想简单的自定义某个类的单个属性访问的话就
不用去写描述器了。这种情况下使用 8.6 小节介绍的 property 技术会更加容易。当程
序中有很多重复代码的时候描述器就很有用了 (比如你想在你代码的很多地方使用描述
器提供的功能或者将它作为一个函数库特性)

## 简化数据结构的初始化

你写了很多仅仅用作数据结构的类，不想写太多烦人的 __init__() 函数

可以在一个基类中写一个公用的 __init__() 函数

In [26]:
import math

class Structure:
    #class variable that specifies expected fields
    _fields = []
    
    def __init__(self, *args):
        if len(args) != len(self._fields):
            raise TypeError('Expected {} argument'.format(len(self._fields)))
        # set the argument
        for name, value in zip(self._fields, args):
            setattr(self, name, value)

In [27]:
class Stock(Structure):
    _fields = ['name', 'shares', 'prices']
    
class Point(Structure):
    _fields = ['x', 'y']
    
class Circle(Structure):
    _fields = ['radius']
    
    def area(self):
        return math.pi * self.radius ** 2

In [28]:
#use these case

s = Stock('ACME', 50, 91.1)
p = Point(2, 3)
c = Circle(4.5)
s2 = Stock('ACME', 50)

TypeError: Expected 3 argument

In [34]:
# 如果还想支持关键字参数，可以将关键字参数设置为实例属性
class Structure2:
    _fields = []
    
    def __init__(self, *args, **kwargs):
        if len(args) > len(self._fields):
            raise TypeError('Expected {} argument'.format(len(self._fields)))
            
        for name, value in zip(self._fields, args):
            setattr(self, name, value)
            
        for name in self._fields[len(args):]:
            setattr(self, name, kwargs.pop(name))
            
        if kwargs:
            raise TypeError('Invalid argument(s) : {}'.format(','.join(kwargs)))
            
if __name__ == '__main__':
    class Stock(Structure2):
        _fields = ['name', 'shares', 'price']
        
    s1 = Stock('ACME', 50, 91.1)
    s2 = Stock('ACME', 50 , price=91.1)
    s3 = Stock('ACME', shares=59, price=91.1)

你还能将不在 _fields 中的名称加入到属性中去：

In [37]:
class Structure3:
    _fields = []
    
    def __init__(self, *args, **kwargs):
        if len(args) != len(self._fields):
            raise TypeError
            
        for name, value in zip(self._fields, args):
            setattr(self, name, value) # setattr(object, name, value) 
            
        extra_args = kwargs.keys() - self._fields
        for name in extra_args:
            setattr(self, name, kwargs.pop(name))
            
        if kwargs:
            raise TypeError
            
if __name__ == '__main__':
    class Stock(Structure3):
        _fields = ['name', 'shares', 'price']
        
    s1 = Stock('ACME', 50, 91.1)
    s2 = Stock('ACME', 50, 91.1, date='8/2/2019')

当你需要使用大量很小的数据结构类的时候，相比手工一个个定义 __init__()方法而已，使用这种方式可以大大简化代码。

在上面的实现中我们使用了 setattr() 函数类设置属性值，你可能不想用这种方式，而是想直接更新实例字典，就像下面这样：

In [40]:
class Structure:
    _fields = []
    def __init__(self, *args):
        if len(args) != len(self._fields):
            raise TypeError
            
        self.__dict__.update(zip(self._fields, args)) # zip() is not dict update?

尽管这也可以正常工作，但是当定义子类的时候问题就来了。当一个子类定义了
__slots__ 或者通过 property(或描述器) 来包装某个属性，那么直接访问实例字典就
不起作用了。我们上面使用 setattr() 会显得更通用些，因为它也适用于子类情况

## 定义接口或者抽象基类

你想定义一个接口或抽象类，并且通过执行类型检查来确保子类实现了某些特定的方法

使用 abc 模块可以很轻松的定义抽象基类

In [49]:
from abc import ABCMeta, abstractmethod

class IStream(metaclass=ABCMeta):
    @abstractmethod
    def read(self, maxbytes=-1):
        pass
    
    @abstractmethod
    def write(self, data):
        pass

抽象类的一个特点是它不能直接被实例化，比如你想像下面这样做是不行的

In [51]:
a = IStream()

TypeError: Can't instantiate abstract class IStream with abstract methods read, write

抽象类的目的就是让别的类继承它并实现特定的抽象方法：

In [52]:
class SocketSteam(IStream):
    def read(self, maxbytes=-1):
        pass
    
    def write(self, data):
        pass

抽象基类的一个主要用途是在代码中检查某些类是否为特定类型，实现了特定接口：

In [53]:
def serialize(obj, stream):
    if not isinstance(stream, IStream):
        raise TypeError
    pass

除了继承这种方式外，还可以通过注册方式来让某个类实现抽象基类

import io

IStream.register(io.IOBase)

f=open('foo.txt')
isinstance(f, IStream)

@abstractmethod 还能注解静态方法、类方法和 properties 。你只需保证这个注
解紧靠在函数定义前即可

In [55]:
class A(metaclass=ABCMeta):
    @property
    @abstractmethod
    def name(self):
        pass
    @name.setter
    @abstractmethod
    def name(self, value):
        pass
    @classmethod
    @abstractmethod
    def method1(cls):
        pass
    @staticmethod
    @abstractmethod
    def method2():
        pass

标准库中有很多用到抽象基类的地方。collections 模块定义了很多跟容器和迭代器 (序列、映射、集合等) 有关的抽象基类。numbers 库定义了跟数字对象 (整数、浮点数、有理数等) 有关的基类。io 库定义了很多跟 I/O 操作相关的基类。你可以使用预定义的抽象类来执行更通用的类型检查，例如：

In [ ]:
###### import collections
# Check if x is a sequence
if isinstance(x, collections.Sequence):
...
# Check if x is iterable
if isinstance(x, collections.Iterable):
...
# Check if x has a size
if isinstance(x, collections.Sized):
...
# Check if x is a mapping
if isinstance(x, collections.Mapping):

尽管 ABCs 可以让我们很方便的做类型检查，但是我们在代码中最好不要过多的使用它。因为 Python 的本质是一门动态编程语言，其目的就是给你更多灵活性，强制类型检查或让你代码变得更复杂，这样做无异于舍本求末

......

## 实现数据模型的类型约束

你想定义某些在属性赋值上面有限制的数据结构

在这个问题中，你需要在对某些实例属性赋值时进行检查。所以你要自定义属性赋值函数，这种情况下最好使用描述器

## 8.19 实现状态对象或者状态机

你想实现一个状态机或者是在不同状态下执行操作的对象，但是又不想在代码中出现太多的条件判断语句

在很多程序中，有些对象会根据状态的不同来执行不同的操作。比如考虑如下的一个连接对象

In [33]:
class Connection:
    """ 普通方案，好多个判断语句，效率低下~~"""
    def __init__(self):
        self.state = 'closed'
        
    def read(self):
        if self.state !=  'open':
            raise RuntimeError('Not open')
        print('reading')
        
    def write(self, data):
        if self.state != 'open':
            raise RuntimeError('Not open')
        print('writing')
        
    def open(self):
        if self.state == 'open':
            raise RuntimeError('Already open')
        self.state = 'open'
        
    def close(self):
        if self.state == 'closed':
            raise RuntimeError('Already closed')
        self.state = 'closed'

这样写有很多缺点，首先是代码太复杂了，好多的条件判断。其次是执行效率变低，因为一些常见的操作比如 read()、write() 每次执行前都需要执行检查。

一个更好的办法是为每个状态定义一个对象

In [34]:
class Connection1:
    """ 新方案——对每个状态定义一个类"""
    def __init__(self):
        self.new_state(ClosedConnectionState)
        
    def new_state(self, newstate):
        self._state = newstate
        
    def read(self):
        return self._state.read(self)
    
    def write(self):
        return self._state.write(self, data)
    
    def open(self):
        return self._state.open(self)
    
    def close(self):
        return self._state.close(self)

In [35]:
# Connection state base class
class ConnectionState:
    @staticmethod # ?
    def read(conn):
        raise NotImplementedError()
        
    @staticmethod
    def write(conn, data):
        raise NotImplementedError()
        
    @staticmethod
    def open(conn):
        raise NotImplementedError()
        
    @staticmethod
    def close(conn):
        raise NotImplementedError()
        
# Implementation of different states
class ClosedConnectionState(ConnectionState):
    @staticmethod
    def read(conn):
        raise RuntimeError('Not open')
    
    @staticmethod
    def write(conn, data):
        raise RuntimeError('Not open')
    @staticmethod
    def open(conn):
        conn.new_state(OpenConnectionState)
    @staticmethod
    def close(conn):
        raise RuntimeError('Already closed')

In [36]:
class OpenConnectionState(ConnectionState):
    @staticmethod
    def read(conn):
        print('reading')
    @staticmethod
    def write(conn, data):
        print('writing')
    @staticmethod
    def open(conn):
        raise RuntimeError('Already open')
    @staticmethod
    def close(conn):
        conn.new_state(ClosedConnectionState)

In [37]:
c = Connection()
c.state

'closed'

In [38]:
c.read()

RuntimeError: Not open

In [39]:
c.open()

In [40]:
c.state

'open'

In [41]:
c.read() 

reading


In [42]:
c.write('hello')

writing


In [43]:
c.close()

In [44]:
c.state

'closed'

......

如果代码中出现太多的条件判断语句的话，代码就会变得难以维护和阅读。这里的
解决方案是将每个状态抽取出来定义成一个类。

这里看上去有点奇怪，每个状态对象都只有静态方法，并没有存储任何的实例
属性数据。实际上，所有状态信息都只存储在 Connection 实例中。在基类中定义的
NotImplementedError 是为了确保子类实现了相应的方法。这里你或许还想使用 8.12
小节讲解的抽象基类方式。

设计模式中有一种模式叫状态模式，这一小节算是一个初步入门！

## 8.20 通过字符串调用对象方法

你有一个字符串形式的方法名称，想通过它调用某个对象的对应方法。

最简单的情况，可以使用 getattr() ：

In [7]:
import math

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return 'Point({!r:}, {!r:})'.format(self.x, self.y)
    
    def distance(self, x, y):
        return math.hypot(self.x - x, self.y - y)

p = Point(2, 3)
d = getattr(p, 'distance')(0, 0)

In [8]:
d

3.605551275463989

In [9]:
p.distance(0, 0)

3.605551275463989

另外一种方法是使用 operator.methodcaller() ，例如：

In [11]:
import operator

operator.methodcaller('distance', 0, 0)(p)

3.605551275463989

当你需要通过相同的参数多次调用某个方法时，使用 operator.methodcaller 就很方便了。比如你需要排序一系列的点，就可以这样做：

In [13]:
points = [
    Point(1, 2),
    Point(3, 0),
    Point(10, -3),
    Point(-5, -7),
    Point(-1, 8),
    Point(3, 2) 
]

# Sort by distance from orign(0, 0)
points.sort(key=operator.methodcaller('distance', 0, 0))

In [14]:
points

[Point(1, 2),
 Point(3, 0),
 Point(3, 2),
 Point(-1, 8),
 Point(-5, -7),
 Point(10, -3)]

调用一个方法实际上是两部独立操作，第一步是查找属性，第二步是函数调用。因
此，为了调用某个方法，你可以首先通过 getattr() 来查找到这个属性，然后再去以
函数方式调用它即可。

operator.methodcaller() 创建一个可调用对象，并同时提供所有必要参数，然
后调用的时候只需要将实例对象传递给它即可，比如：

In [17]:
p = Point(3, 4)
d = operator.methodcaller('distance', 0, 0)
d(p)

8.246211251235321

In [19]:
p = Point(3, 4)
d = operator.methodcaller('distance', 30, 40)
d(p)

45.0

通过方法名称字符串来调用方法通常出现在需要模拟 case 语句或实现访问者模式
的时候。参考下一小节获取更多高级例子

## 8.21 实现访问者模式

你要处理由大量不同类型的对象组成的复杂数据结构，每一个对象都需要需要进
行不同的处理。比如，遍历一个树形结构，然后根据每个节点的相应状态执行不同的操
作。